In [ ]:
USE ROLE CORTEX_USER_ROLE;
USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Fact table: daily_revenue
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.daily_revenue (
    date DATE,
    revenue FLOAT,
    cogs FLOAT,
    forecasted_revenue FLOAT,
    product_id INT,
    region_id INT
);

In [ ]:
USE ROLE CORTEX_USER_ROLE;
USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Dimension table: product_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.product_dim (
    product_id INT,
    product_line VARCHAR(16777216)
);

In [ ]:
USE ROLE CORTEX_USER_ROLE;
USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Dimension table: region_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.region_dim (
    region_id INT,
    sales_region VARCHAR(16777216),
    state VARCHAR(16777216)
);

**Importing Libraries**
- [Snowpark Pandas](https://docs.snowflake.com/developer-guide/snowpark/python/pandas-on-snowflake?_fsi=10ufziXM&_fsi=10ufziXM&_fsi=10ufziXM#how-pandas-on-snowflake-compares-to-native-pandas) lets developers run their pandas code directly on their data in Snowflake. Users will be able to get the same pandas-native experience they know and love with Snowflake's performance, scale and governance.The Snowpark pandas API is available as part of the Snowpark Python package (version 1.17 and above). Snowpark Python comes pre-installed with the Snowflake Notebooks environment. Additionally, you will need to add the modin package in the Packages dropdown.


- [Streamlit](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-visualize-data#label-notebooks-visualize-cells-streamlit) Streamlit is an open-source Python library that makes it easy to create and share custom web apps for machine learning, data science and analytics. By using Streamlit you can quickly build and deploy powerful data applications. For more information about the open-source library, see the [Streamlit Library documentation](https://docs.streamlit.io/).

- [Matplotlib](https://matplotlib.org/) Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python.

In [ ]:
# Import the Snowpark pandas plugin for modin
import snowflake.snowpark.modin.plugin
import modin.pandas as pd

import matplotlib.pyplot as plt
import streamlit as st
import matplotlib.ticker as ticker
import matplotlib.dates as mdates

# Import Snowpark session
from snowflake.snowpark.context import get_active_session


**Getting active Snowflake session and set current schema**

In [ ]:
# Retrieve the Session
session = get_active_session()

# Set context
session.use_schema('cortex_analyst_demo.revenue_timeseries')
session.use_role('CORTEX_USER_ROLE')

**Loading data csv files from raw_data stage**
- daily_revenue.csv
- product.csv
- region.csv

In [ ]:
df_d_rev = pd.read_csv('@RAW_DATA/daily_revenue.csv')
df_prod = pd.read_csv('@RAW_DATA/product.csv')
df_reg = pd.read_csv('@RAW_DATA/region.csv')

**Ploting Total revenue by Product Id for all time**

In [ ]:
df_region_sum = df_d_rev.groupby('Product_id')['REVENUE'].sum()

ax = df_region_sum.plot.bar()
ax.set_xlabel('Product Id')
ax.set_ylabel('Total Revenue')

formatter = ticker.StrMethodFormatter('${x:,.0f}')
ax.yaxis.set_major_formatter(formatter)

plt.show()

**Ploting Total revenue by Product Name for all time**

In [ ]:
# Join the the product dataset with daily revenue data set to get Product Names
df_total_p_line_rev = df_d_rev.join(df_prod.set_index('Product_id'), 
                                    on='Product_id').groupby('Product_line')['REVENUE'].sum()

ax = df_total_p_line_rev.plot.bar()
ax.set_xlabel('Product')
ax.set_ylabel('Total Revenue')

formatter = ticker.StrMethodFormatter('${x:,.0f}')
ax.yaxis.set_major_formatter(formatter)

plt.show()

**Show the difference in actual vs forecasted revenue for the first 10 records**

In [ ]:
df_d_rev_sub = df_d_rev['DATE', 'REVENUE', 'FORECASTED_REVENUE']

df_d_rev_sub['DIFF'] =  df_d_rev_sub['REVENUE'] - df_d_rev_sub['FORECASTED_REVENUE']

# Take last 10 records
df_d_rev_sub_10 = df_d_rev_sub.set_index('DATE').head(10)

ax = df_d_rev_sub_10['DIFF'].plot.bar()

formatter_date = mdates.DateFormatter('%m/%d')
ax.xaxis.set_major_formatter(formatter_date)

formatter = ticker.StrMethodFormatter('${x:,.0f}')
ax.yaxis.set_major_formatter(formatter)

ax.set_xlabel('Date')
ax.set_ylabel('Difference in Revenue vs Forecast')

plt.show()

**We are going to use to_snowflake() function to save our dataset to tables in Snowflake**

In [ ]:
# Save data to region_dim table
df_reg.to_snowflake(
    name = 'cortex_analyst_demo.revenue_timeseries.region_dim',
    if_exists='append',
    index=False
)

# Save data to product_dim table
df_prod.to_snowflake(
    name = 'cortex_analyst_demo.revenue_timeseries.product_dim',
    if_exists='append',
    index=False
)

df_d_rev.to_snowflake(
    name = 'cortex_analyst_demo.revenue_timeseries.daily_revenue',
    if_exists='append',
    index=False
)


**Query daily_revenue table to see first 10 rows**

In [ ]:
select * from cortex_analyst_demo.revenue_timeseries.daily_revenue limit 10;

### Integrate Cortex Search
We will integrate Cortex Search as a way to improve literal string searches to help Cortex Analyst generate more accurate SQL queries.
![Cortex Search](https://docs.snowflake.com/_images/cortex-search-rag.png)

In [ ]:
USE ROLE CORTEX_USER_ROLE;
USE DATABASE cortex_analyst_demo;
USE SCHEMA revenue_timeseries;

  CREATE OR REPLACE CORTEX SEARCH SERVICE product_line_search_service
  ON product_dimension
  WAREHOUSE = cortex_analyst_wh
  TARGET_LAG = '1 hour'
  AS (
        SELECT DISTINCT product_line AS product_dimension FROM product_dim
      );

### What is a Sementic Model?
A semantic model is a conceptual data model that describes the meaning and relationships between data elements, essentially translating raw data into a business-friendly structure by defining how data is related to the real world, allowing for easier analysis and interpretation.

### Why use Semantic Models?
Cortex Analyst allows users to ask questions about Snowflake data using natural language. Typically there is a gap between the vocabulary used by business users when asking data questions and the vocabulary used in the database schema. These users typically use business or domain specific terms, whereas the database schema may use abbreviations or terms often used in ETL pipelines.

![Cortex Analyst](https://raw.githubusercontent.com/fahadaz/cortex_analyst_hol/refs/heads/main/images/cortex_analyst_architecture.jpeg)


### Semantic Model Details
The semantic model file **[revenue_timeseries.yaml](https://github.com/fahadaz/cortex_analyst_hol/blob/main/data/revenue_timeseries.yaml)** is the key that unlocks Cortex Analyst's power. This YAML file dictates the tables, columns, etc. that Analyst can use in order to run queries that answer natural-language questions Let's talk a little about the details of this file:

The Semantic Model is composed of a number of different fields that help Cortex Analyst understand the specifics of your data:

- Logical Tables which are composed of Logical Columns
- Logical Columns which are one of dimensions, time_dimensions, or measures
- Relationships that exist between tables to allow for JOINS

Logical Tables are relatively straightforward- these are tables or views within a database. That's it! Pretty simple

Logical Columns get a bit more complicated; a logical column can reference an underlying physical column in a table, or it can be a expression containing one or more physical columns. So, for example, in the **[revenue_timeseries.yaml](https://github.com/fahadaz/cortex_analyst_hol/blob/main/data/revenue_timeseries.yaml)**, we have a simple logical column **daily_revenue** that is a physical column. In the **daily_revenue** measure definition, you'll notice that we provide a description, as well as synonyms, data_type, and a default_aggregation, but no **expr** parameter. This is because revenue is simply a physical column in the **daily_revenue** table:

```
measures:
    - name: daily_revenue
        expr: revenue
        description: total revenue for the given day
        synonyms: ["sales", "income"]
        default_aggregation: sum
        data_type: number
```
In contrast, we define a different measure daily_profit which is not in fact a physical column, but rather an expression of the difference between the revenue and cogs physical columns:

```
- name: daily_profit
    description: profit is the difference between revenue and expenses.
    expr: revenue - cogs
    data_type: number
```

In the semantic model, time_dimensions specifically capture temporal features of the data, and dimensions are not quantitative fields (e.g. quantitative fields are measures, while categorical fields are dimensions).

An example time_dimension:
```
time_dimensions:
    - name: date
    expr: date
    description: date with measures of revenue, COGS, and forecasted revenue for each product line
    unique: false
    data_type: date
```
An example relationship:

```
relationships:
  - name: revenue_to_product
    left_table: daily_revenue
    right_table: product
    relationship_columns:
      - left_column: product_id
        right_column: product_id
    join_type: left_outer
    relationship_type: many_to_one
```

When generating the semantic model, think from the end user perspective:

- For business user, accuracy and trust is the paramount
- Organize your YAML file in the unit of business domain/topic
- If you are trying to pull a snippet of this data into excel for your business stakeholder, what are the tabs and columns you'd keep? - - - What are the column namings you'd use?
- Use above to guide your selection of tables and columns. Err on the side of only including necessary columns.
- We recommend not exceeding 3-5 tables, 10-20 columns each table to start.

For more information about the semantic model, please refer to the [documentation](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst/semantic-model-spec).

In [ ]:
question = 'What was the total revenue broken by product for all time?'

In [ ]:
"""
Cortex Analyst App
====================
This app allows users to interact with their data using natural language.
"""

import json  # To handle JSON data

import _snowflake  # For interacting with Snowflake-specific APIs
import streamlit as st  # Streamlit library for building the web app
from snowflake.snowpark.context import (
    get_active_session,
) 


API_ENDPOINT = "/api/v2/cortex/analyst/message"
API_TIMEOUT = 50000  # in milliseconds


# Initialize a Snowpark session for executing queries
session = get_active_session()

"""
    Send chat history to the Cortex Analyst API and return the response.

    Args:
        messages (List[Dict]): The conversation history.

    Returns:
        Optional[Dict]: The response from the Cortex Analyst API.
"""
# Prepare the request body with the user's prompt
request_body = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": question
                }
            ]
        },
    ],
    "semantic_model_file": "@CORTEX_ANALYST_DEMO.REVENUE_TIMESERIES.RAW_DATA/revenue_timeseries.yaml",
}

# Send a POST request to the Cortex Analyst API endpoint
# Adjusted to use positional arguments as per the API's requirement
resp = _snowflake.send_snow_api_request(
        "POST",  # method
        API_ENDPOINT,  # path
        {},  # headers
        {},  # params
        request_body,  # body
        None,  # request_guid
        API_TIMEOUT,  # timeout in milliseconds
    )

# Content is a string with serialized JSON object
parsed_content = json.loads(resp["content"])

st.json(parsed_content)

In [ ]:
WITH __daily_revenue AS (
  SELECT
    product_id,
    revenue AS daily_revenue
  FROM cortex_analyst_demo.revenue_timeseries.daily_revenue
), __product AS (
  SELECT
    product_id,
    product_line
  FROM cortex_analyst_demo.revenue_timeseries.product_dim
)
SELECT
  p.product_line,
  SUM(dr.daily_revenue) AS total_revenue
FROM __daily_revenue AS dr
LEFT OUTER JOIN __product AS p
  ON dr.product_id = p.product_id
GROUP BY
  p.product_line
ORDER BY
  total_revenue DESC NULLS LAST
 -- Generated by Cortex Analyst
;